<img src = "https://8weeksqlchallenge.com/images/case-study-designs/2.png">

## Introduction

Did you know that over 115 million kilograms of pizza is consumed daily worldwide??? (Well according to Wikipedia anyway…)

Danny was scrolling through his Instagram feed when something really caught his eye - “80s Retro Styling and Pizza Is The Future!”

Danny was sold on the idea, but he knew that pizza alone was not going to help him get seed funding to expand his new Pizza Empire - so he had one more genius idea to combine with it - he was going to Uberize it - and so Pizza Runner was launched!

## Problem Statement

Danny started by recruiting “runners” to deliver fresh pizza from Pizza Runner Headquarters (otherwise known as Danny’s house) and also maxed out his credit card to pay freelance developers to build a mobile app to accept orders from customers.


## Entity Relationship Diagram

<img src = "https://user-images.githubusercontent.com/81607668/127271531-0b4da8c7-8b24-4a14-9093-0795c4fa037e.png">

In [1]:
import sqlite3
import pandas as pd

In [2]:
database = '../data/PizzaRunner.sqlite'

In [3]:
connection = sqlite3.connect(database)

# Case Study Questions

---

## B. Runner and Customer Experience


---

### 1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [4]:
pd.read_sql("""

            SELECT 
              (cast(strftime('%d', registration_date) as int) / 7) + 1 AS "Registration Week",
              COUNT(runner_id) AS "Runner Signup"
            FROM runners
            GROUP BY "Registration Week";
            
            """,connection)

,Registration Week,Runner Signup
0,1,2
1,2,1
2,3,1


### 2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [5]:
pd.read_sql("""

            WITH time_taken_cte AS
            (
              SELECT 
                c.order_id, 
                c.order_time, 
                r.pickup_time, 
                ROUND(((JULIANDAY(r.pickup_time) - JULIANDAY(c.order_time))* 1440)) AS pickup_minutes
              FROM customer_orders AS c
              JOIN runner_orders AS r
                ON c.order_id = r.order_id
              WHERE r.distance != 0
              GROUP BY c.order_id, c.order_time, r.pickup_time
            )

            SELECT 
              ROUND(AVG(pickup_minutes)) AS "Avg Pickup Mins"
            FROM time_taken_cte
            WHERE pickup_minutes > 1;

""",connection)

,Avg Pickup Mins
0,16.0


### 3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [10]:
pd.read_sql("""

            WITH prep_time AS
            (
              SELECT 
                c.order_id, 
                COUNT(c.order_id) AS pizza_order, 
                c.order_time, 
                r.pickup_time, 
                ROUND(((JULIANDAY(r.pickup_time) - JULIANDAY(c.order_time))* 1440)) AS prep_minutes
              FROM customer_orders AS c
              JOIN runner_orders AS r
                ON c.order_id = r.order_id
              WHERE r.distance != 0
              GROUP BY c.order_id, c.order_time, r.pickup_time
            )

            SELECT 
              pizza_order AS "Pizza Order", 
              AVG(prep_minutes) AS "Avg Prep Time (Mins)"
            FROM prep_time
            WHERE prep_minutes > 1
            GROUP BY "Pizza Order";
            
            """,connection)

,Pizza Order,Avg Prep Time (Mins)
0,1,12.2
1,2,18.5
2,3,29.0


### 4. What was the average distance travelled for each customer?

In [11]:
pd.read_sql("""

            SELECT 
              c.customer_id AS Customer, 
              ROUND(AVG(r.distance)) AS "Avg Distance"
            FROM customer_orders AS c
            JOIN runner_orders AS r
              ON c.order_id = r.order_id
            WHERE r.duration != 0
            GROUP BY Customer;

            """,connection)

,Customer,Avg Distance
0,101,20.0
1,102,17.0
2,103,23.0
3,104,10.0
4,105,25.0


### 5. What was the difference between the longest and shortest delivery times for all orders?

In [12]:
pd.read_sql("""

            SELECT MAX(duration) - MIN(duration) AS "Delivery Time Difference"
            FROM runner_orders
            WHERE duration NOT LIKE ' ';
            
            """,connection)

,Delivery Time Difference
0,30.0


### 6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

In [13]:
pd.read_sql("""
            
            SELECT 
              r.runner_id AS "Runner ID", 
              c.customer_id AS "Customer ID", 
              c.order_id AS "Order ID", 
              COUNT(c.order_id) AS "Pizza Count", 
              r.distance AS Distance, 
              (r.duration / 60) AS "Duration Hr" , 
              ROUND((r.distance/r.duration * 60), 2) AS "Avg speed"
            FROM runner_orders AS r
            JOIN customer_orders AS c
              ON r.order_id = c.order_id
            WHERE distance != 0
            GROUP BY "Runner ID", "Customer ID", "Order ID", Distance, r.duration
            ORDER BY "Customer ID";

            """,connection)

,Runner ID,Customer ID,Order ID,Pizza Count,Distance,Duration Hr,Avg speed
0,1,101,1,1,20.0,0.533333,37.50
1,1,101,2,1,20.0,0.450000,44.44
2,1,102,3,2,13.4,0.333333,40.20
3,2,102,8,1,23.4,0.250000,93.60
4,2,103,4,3,23.4,0.666667,35.10
5,1,104,10,2,10.0,0.166667,60.00
6,3,104,5,1,10.0,0.250000,40.00
7,2,105,7,1,25.0,0.416667,60.00


### 7.  What is the successful delivery percentage for each runner?

In [14]:
pd.read_sql("""

            SELECT 
              runner_id AS Runner, 
              ROUND(100 * SUM(
                CASE WHEN distance != 0 THEN 1
                ELSE 0 END) / COUNT(*), 0) AS "Success Percentage"
            FROM runner_orders
            GROUP BY runner_id;


            """,connection)

,Runner,Success Percentage
0,1,100.0
1,2,75.0
2,3,50.0
